# COMP9417 Assignment
## Topick: Recommendation Sytem

**Project Description:**
The goal of the project is to make a **Movie Recommendation System** with the dataset from GroupLens: https://grouplens.org/datasets/movielens/. By split the data to train dataset and test dataset, the work is to make a prediction about the movie that the user would be liked. Firstly, we will try 10MB dataset and evaluate the result and then try larger dataset and evalate the result. 


## Process
1. data cleaning
2. decide recommendation algorithm
3. model
4. analyse

## 1. Data Cleaning

In [3]:
!ls data/ml-100k/

README       u.genre      u.user       u2.test      u4.test      ua.test
allbut.pl    u.info       u1.base      u3.base      u5.base      ub.base
mku.sh       u.item       u1.test      u3.test      u5.test      ub.test
u.data       u.occupation u2.base      u4.base      ua.base


In [11]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as pl
# import sys
# reload(sys)
# sys.setdefaultencoding("utf-8")

In [40]:
Udata_header = ['user_id', 'item_id', 'rating', 'timestamp']
m_cols = ['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url']

Udata_df = pd.read_csv('data/ml-100k/u.data',sep='\t',names=Udata_header)
# Uitem_header = ['item_id', 'movie_title', 'release_date', 'video_release_date',
#                 'IMDb_URL', 'unknown', 'Action', 'Adventure' ,'Animation', 
#               'Childrens','Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
#               'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
#               'Thriller', 'War', 'Western']
Uitem_df = pd.read_csv('data/ml-100k/u.item',sep='|',names=m_cols,encoding='latin1',usecols=range(5))  #encoding = "ISO-8859-1"

Ugenre_header = ['Type','id']
Ugenre_df = pd.read_csv('data/ml-100k/u.genre',sep='|',names=Ugenre_header)

Uuser_header = ['user_id', 'age', 'gender', 'occupation' ,'zip_code']
Uuser_df = pd.read_csv('data/ml-100k/u.user',sep='|',names=Uuser_header)


## Generate the final Dataframe
Group the Dataframe by the user_id

In [47]:
Total_df = pd.merge(Udata_df,Uitem_df,on = "item_id") 
Total_df = pd.merge(Total_df,Uuser_df,on = "user_id")
groupbyUser=Total_df.groupby(["user_id"])
groupbyUser.head(20)


,user_id,item_id,rating,timestamp,title,release_date,video_release_date,imdb_url,age,gender,occupation,zip_code
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),49,M,writer,55105
1,196,257,2,881251577,Men in Black (1997),04-Jul-1997,NaN,http://us.imdb.com/M/title-exact?Men+in+Black+...,49,M,writer,55105
2,196,111,4,881251793,"Truth About Cats & Dogs, The (1996)",26-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Truth%20About...,49,M,writer,55105
3,196,25,4,881251955,"Birdcage, The (1996)",08-Mar-1996,NaN,"http://us.imdb.com/M/title-exact?Birdcage,%20T...",49,M,writer,55105
4,196,382,4,881251843,"Adventures of Priscilla, Queen of the Desert, ...",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Adventures%20...,49,M,writer,55105
5,196,202,3,881251728,Groundhog Day (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Groundhog%20D...,49,M,writer,55105
6,196,153,5,881251820,"Fish Called Wanda, A (1988)",01-Jan-1988,NaN,http://us.imdb.com/M/title-exact?Fish%20Called...,49,M,writer,55105
7,196,286,5,881250949,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,49,M,writer,55105
8,196,66,3,881251911,While You Were Sleeping (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?While%20You%2...,49,M,writer,55105
9,196,845,4,881251954,That Thing You Do! (1996),28-Sep-1996,NaN,http://us.imdb.com/M/title-exact?That%20Thing%...,49,M,writer,55105


# 2. Decide Recommendation Algorithm
Before the decision of Recommendation Algorithm, we need to define the Target Effect of the recommendation system.

**Target Effect Definition:**
According to the dataset, there are **n** numbers of users and **m** numbers of movies, each user may have seen the movie, and give their ratings.
Based on the dataset of the user and their rating to the movie, what we want to do is to find the similarity of the user by the similarity of the rating to the movie. If they both give the relative high score to the same movie, they are similar, and so on. Based on this, if the result is that user A is similar to user B, and what A have seen in his movie list and the movie that is given high score, that movie will recommend to user B.



**There are several options to use in recommendation system**
1. Content-based, collaborative and hybrid recommender systems
2. reproduce key similarity-based approaches to recommender systems

**Algorithm Decision:**
According to the goal what we want to do, similarity-based approaches to recommender systems is the best choice.

# 3. Modelling

In [51]:
import skitlearn

ModuleNotFoundError: No module named 'skitlearn'